# Reading Data

In [10]:
import json

path = "/home/drissdo/Desktop/Law_Q&A/data/structured_data_doc.json"
with open(path) as f:
    all_data = json.load(f)

len(all_data)

5455

In [11]:
# all_data[2]

# Loading data

In [12]:
## lOADL 

from llama_index.core import Document

documents = []
count = 0
for doc_data in all_data:
    for child_data in doc_data["child_data"]:
        if len(child_data["lower_segmented_text"].strip()): # Check if empty or not
            documents.append(Document(text=child_data["full_text"],
                                    metadata={
                                        "filename": doc_data["meta_data"]["file_name"],
                                        "source": doc_data["meta_data"]["id_doc"] + ", " + ", ".join(child for child in child_data['pointer_link']),
                                        "len_tokenizer": child_data["len_tokenizer"]
                                    },
                                    text_template="{content}",
                                    excluded_embed_metadata_keys=["filename", "source", "len_tokenizer"],
                                    excluded_llm_metadata_keys=["filename", "source", "len_tokenizer"]))
        else:
            count += 1

print(f"There are {count} empty documents")

There are 0 empty documents


In [13]:
documents[0].metadata

{'filename': 'Decree_No._139_2021_ND-CP_dated_December_31,_2021_.json',
 'source': '139/2021/ND-CP, Chương I, Điều 1',
 'len_tokenizer': 157}

# Transformation

In [14]:
from pyvi import ViTokenizer
from transformers import AutoTokenizer
from llama_index.core.node_parser import TokenTextSplitter, SentenceSplitter

tokenizer = AutoTokenizer.from_pretrained("bkai-foundation-models/vietnamese-bi-encoder")

/home/drissdo/anaconda3/envs/thesis/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
base_node_parser = TokenTextSplitter( 
                                chunk_overlap=0,
                                chunk_size=900,
                                separator=" ",
                                backup_separators=["__", "..", "--"],
                                include_prev_next_rel=False
                                )


base_nodes = base_node_parser.get_nodes_from_documents(documents,
                                                       show_progress=True)

Parsing nodes: 100%|██████████| 142362/142362 [04:13<00:00, 561.88it/s] 


In [16]:
len(base_nodes), base_nodes[0]

(177893,
 TextNode(id_='f275d313-0c46-4b2c-b5f6-58fed0959871', embedding=None, metadata={'filename': 'Decree_No._139_2021_ND-CP_dated_December_31,_2021_.json', 'source': '139/2021/ND-CP, Chương I, Điều 1', 'len_tokenizer': 157}, excluded_embed_metadata_keys=['filename', 'source', 'len_tokenizer'], excluded_llm_metadata_keys=['filename', 'source', 'len_tokenizer'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='646e04c3-7d3d-401b-b914-b693eb007ff9', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'filename': 'Decree_No._139_2021_ND-CP_dated_December_31,_2021_.json', 'source': '139/2021/ND-CP, Chương I, Điều 1', 'len_tokenizer': 157}, hash='e4fa2847d6f6a6f3fbf15130e07caf1867c5e72fa36feb8b6e66895f51c9efb1')}, text='Điều 1. Phạm vi điều chỉnh\n1. Nghị định này quy định về:\n\ta) Hành vi vi phạm hành chính, hình thức xử phạt, mức xử phạt, biện pháp khắc phục hậu quả và thẩm quyền lập biên bản vi phạm hành chính, thẩm quyền xử phạt vi phạm hành chính trong lĩnh vực g

In [17]:
# # Save as parent parquet 
# import os
# from datasets import Dataset

# dataset = Dataset.from_list([node.to_dict() for node in base_nodes])

# with open(os.path.join("/home/s/ducnq/law-rag/data/hf", "parent_nodes.parquet"), "wb") as fOut:
#     dataset.to_parquet(fOut)

In [18]:
# Child nodes
from tqdm import tqdm
for base_node in tqdm(base_nodes):
    base_node.metadata["parent_text"] = base_node.text
    # base_node.text = ViTokenizer.tokenize(base_node.text.lower())
    base_node.excluded_embed_metadata_keys.append("parent_text")
    base_node.excluded_llm_metadata_keys.append("parent_text")

100%|██████████| 177893/177893 [00:00<00:00, 1434322.29it/s]


In [19]:
child_node_parser= SentenceSplitter(
                    chunk_size=600,
                    chunk_overlap=90,
                    separator=" ",
                    include_prev_next_rel=False,
                    )

In [20]:
child_nodes = child_node_parser.get_nodes_from_documents(base_nodes,
                                                         show_progress=True)

Parsing nodes: 100%|██████████| 177893/177893 [01:37<00:00, 1818.00it/s]


In [21]:
len(child_nodes), child_nodes[2]

(239561,
 TextNode(id_='0fc7834a-cdc8-4829-ad5e-8f80df20337c', embedding=None, metadata={'filename': 'Decree_No._139_2021_ND-CP_dated_December_31,_2021_.json', 'source': '139/2021/ND-CP, Chương I, Điều 2', 'len_tokenizer': 426, 'parent_text': 'Điều 2. Đối tượng áp dụng\n1. Cá nhân, tổ chức Việt Nam và nước ngoài có hành vi vi phạm hành chính trong lĩnh vực giao thông đường thủy nội địa tại Việt Nam.\n2. Người có thẩm quyền lập biên bản, thẩm quyền xử phạt vi phạm hành chính và tổ chức, cá nhân có liên quan đến việc xử phạt vi phạm hành chính theo quy định tại Nghị định này.\n3. Tổ chức quy định tại khoản 1 Điều này gồm:\n\ta) Tổ chức kinh tế được thành lập theo quy định của Luật Doanh nghiệp gồm: Doanh nghiệp tư nhân, công ty cổ phần, công ty trách nhiệm hữu hạn, công ty hợp danh;\n\tb) Tổ chức kinh tế được thành lập theo quy định của Luật Hợp tác xã gồm: Hợp tác xã, liên hiệp hợp tác xã;\n\tc) Tổ chức xã hội, tổ chức chính trị-xã hội, tổ chức xã hội - nghề nghiệp;\n\td) Đơn vị sự nghi

In [ ]:
from tqdm import tqdm
from llama_index.core.schema import NodeRelationship

for child_node in tqdm(child_nodes):
    child_node.text = ViTokenizer.tokenize(child_node.text.lower())
    try:
        del child_node.relationships[NodeRelationship.SOURCE].metadata
    except AttributeError:
        continue

  7%|▋         | 15607/239561 [00:29<06:25, 580.53it/s]

In [15]:
len(child_nodes), child_nodes[2]

(239561,
 TextNode(id_='d7d314aa-728e-4cae-bbc9-b3de542e373a', embedding=None, metadata={'filename': 'Decree_No._139_2021_ND-CP_dated_December_31,_2021_.json', 'source': '139/2021/ND-CP, Chương I, Điều 2', 'len_tokenizer': 426, 'parent_text': 'Điều 2. Đối tượng áp dụng\n1. Cá nhân, tổ chức Việt Nam và nước ngoài có hành vi vi phạm hành chính trong lĩnh vực giao thông đường thủy nội địa tại Việt Nam.\n2. Người có thẩm quyền lập biên bản, thẩm quyền xử phạt vi phạm hành chính và tổ chức, cá nhân có liên quan đến việc xử phạt vi phạm hành chính theo quy định tại Nghị định này.\n3. Tổ chức quy định tại khoản 1 Điều này gồm:\n\ta) Tổ chức kinh tế được thành lập theo quy định của Luật Doanh nghiệp gồm: Doanh nghiệp tư nhân, công ty cổ phần, công ty trách nhiệm hữu hạn, công ty hợp danh;\n\tb) Tổ chức kinh tế được thành lập theo quy định của Luật Hợp tác xã gồm: Hợp tác xã, liên hiệp hợp tác xã;\n\tc) Tổ chức xã hội, tổ chức chính trị-xã hội, tổ chức xã hội - nghề nghiệp;\n\td) Đơn vị sự nghi

In [16]:
child_nodes[0].to_dict()

/home/drissdo/anaconda3/envs/thesis/lib/python3.9/site-packages/llama_index/core/schema.py:79: UserWarning: Pydantic serializer warnings:
  Expected `Union[definition-ref, list[definition-ref]]` but got `RelatedNodeInfo` with value `RelatedNodeInfo(node_id='...6feb8b6e66895f51c9efb1')` - serialized value may not be as expected
  data = handler(self)


{'id_': 'edf296b6-c206-430b-88d5-01abe306f134',
 'embedding': None,
 'metadata': {'filename': 'Decree_No._139_2021_ND-CP_dated_December_31,_2021_.json',
  'source': '139/2021/ND-CP, Chương I, Điều 1',
  'len_tokenizer': 157,
  'parent_text': 'Điều 1. Phạm vi điều chỉnh\n1. Nghị định này quy định về:\n\ta) Hành vi vi phạm hành chính, hình thức xử phạt, mức xử phạt, biện pháp khắc phục hậu quả và thẩm quyền lập biên bản vi phạm hành chính, thẩm quyền xử phạt vi phạm hành chính trong lĩnh vực giao thông đường thủy nội địa;\n\tb) Các hành vi vi phạm về hoạt động của phương tiện thủy nội địa ngoài phạm vi luồng và vùng nước chưa được tổ chức quản lý, khai thác giao thông vận tải, bao gồm: Vi phạm của phương tiện thủy nội địa; thuyền viên, người lái phương tiện; quy tắc giao thông và tín hiệu của phương tiện; vận tải đường thủy nội địa; tai nạn giao thông đường thủy nội địa và tìm kiếm, cứu nạn giao thông đường thủy nội địa.\n2. Các hành vi vi phạm hành chính khác liên quan đến giao thông đư

In [17]:
# # Save as child parquet 
# import os
# from datasets import Dataset

# dataset = Dataset.from_list([node.to_dict() for node in child_nodes])

# with open(os.path.join("/home/s/ducnq/law-rag/data/hf", "child_nodes.parquet"), "wb") as fOut:
#     dataset.to_parquet(fOut)

In [18]:
# # Upload to folder to HF 
# from huggingface_hub import HfApi
# 
# REPO_ID = "bkai-foundation-models/TVPL"
# REPO_TYPE = "dataset"
# api = HfApi()
# 
# # api.create_repo(
# #     repo_id=REPO_ID,
# #     private=True,
# #     repo_type=REPO_TYPE,
# #     exist_ok=False
# # )
# 
# 
# api.upload_folder(folder_path="/home/s/ducnq/law-rag/data/hf",
#                   repo_id=REPO_ID,
#                   repo_type=REPO_TYPE)

In [19]:
# from llama_index.llms.llama_cpp.llama_utils import messages_to_prompt 

# Indexing

In [20]:
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.vector_stores.chroma import ChromaVectorStore

MODEL_NAME = "bkai-foundation-models/vietnamese-bi-encoder"
embed_model = HuggingFaceEmbedding(model_name=MODEL_NAME, 
                                   max_length=256)

vector_store = ChromaVectorStore(path="./chroma_db")

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(child_nodes, 
                         storage_context=storage_context, 
                         embed_model=embed_model,
                         insert_batch_size=32768,
                         show_progress=True)

Error while downloading from https://cdn-lfs.hf.co/repos/6a/01/6a013f50310eb721451b1ec4fc4c296585a62ff28aed65f2dc482828fc2f5bd8/e681accadaec87e79901db0c3f68e33d996cba334633b6dd0b2483dba4f398e0?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1729831321&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyOTgzMTMyMX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy82YS8wMS82YTAxM2Y1MDMxMGViNzIxNDUxYjFlYzRmYzRjMjk2NTg1YTYyZmYyOGFlZDY1ZjJkYzQ4MjgyOGZjMmY1YmQ4L2U2ODFhY2NhZGFlYzg3ZTc5OTAxZGIwYzNmNjhlMzNkOTk2Y2JhMzM0NjMzYjZkZDBiMjQ4M2RiYTRmMzk4ZTA%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=qZyAN724OlzqfQW5Xtu9WPtBqonXxh7XZjQwXz0CsT%7E6zVTBL9w8v0--Gk%7EZfdTEZPY0PijLs-GLA1NxkXAL3k8l8Ui40EVtB%7En1eoLt8UX5roNUuIL5L6rwWAkjf7BFk3Rdw41w7kinZATfVdFe8rU5sFYLXy5L1lpttAYzUdom2XkarTRoMv%7EQyTIqSTeoZDI3KfruW-5H6DPd0W830LSRcrZnWqiV2mZIBsoBwPjUOYkG3PIk50cVuX70CpnqbKW

TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'

# Test Retrieval

In [31]:
## 
from pyvi import ViTokenizer
from llama_index.core.response.notebook_utils import display_source_node
base_retriever = index.as_retriever(vector_store_query_mode="hybrid",
                                    similarity_top_k=100, 
                                    alpha=0.5)

In [32]:
TEST_QUESTION = "Mình đi xe máy không đội mũ bảo hiểm bị phạt bao nhiêu tiền?"
retrievals = base_retriever.retrieve(
    ViTokenizer.tokenize(TEST_QUESTION.lower())
)

for n in retrievals[:5]:
    display_source_node(n, source_length=1000, show_source_metadata=True)

**Node ID:** d794f7a7-1fe2-4207-964d-c58a0a3ea032<br>**Similarity:** 0.984487734<br>**Text:** điều 8 . trách_nhiệm của người điều_khiển , người ngồi trên xe mô_tô , xe gắn_máy , xe_đạp máy . người điều_khiển , người ngồi trên xe mô_tô , xe gắn_máy ( kể_cả xe_máy_điện ) , xe_đạp máy khi tham_gia giao_thông có trách_nhiệm : 
 1 . đội mũ bảo_hiểm theo đúng quy_định của pháp_luật . 
 2 . cài quai mũ theo quy_định sau đây : 
 a ) kéo quai mũ bảo_hiểm sang hai bên rồi đội mũ và đóng khóa mũ lại . không được để quai mũ lỏng_lẻo mà phải đóng khít với cằm ; 
 b ) sau khi đội mũ bảo_hiểm cần kiểm_tra lại bằng cách : dùng tay kéo mũ từ đằng sau ra đằng trước hoặc nâng phần trên trước trán ( hoặc phần_cằm đối_với mũ cả hàm ) lên rồi kéo ra đằng sau , mũ không được bật ra khỏi đầu .<br>**Metadata:** {'filename': 'Thông_tư_liên_tịch_06_2013_TTLT-BKHCN-BCT-BCA-BGTV.json', 'source': '06/2013/TTLT-BKHCN-BCT-BCA-BGTVT, Chương 2, Điều 8', 'len_tokenizer': 154, 'parent_text': 'Điều 8. Trách nhiệm của người điều khiển, người ngồi trên xe mô tô, xe gắn máy, xe đạp máy. Người điều khiển, người ngồi trên xe mô tô, xe gắn máy (kể cả xe máy điện), xe đạp máy khi tham gia giao thông có trách nhiệm:\n1. Đội mũ bảo hiểm theo đúng quy định của pháp luật.\n2. Cài quai mũ theo quy định sau đây:\n\ta) Kéo quai mũ bảo hiểm sang hai bên rồi đội mũ và đóng khóa mũ lại. Không được để quai mũ lỏng lẻo mà phải đóng khít với cằm;\n\tb) Sau khi đội mũ bảo hiểm cần kiểm tra lại bằng cách: dùng tay kéo mũ từ đằng sau ra đằng trước hoặc nâng phần trên trước trán (hoặc phần cằm đối với mũ cả hàm) lên rồi kéo ra đằng sau, mũ không được bật ra khỏi đầu.'}<br>

**Node ID:** aa931d77-6922-472c-aae7-3010d9bdb892<br>**Similarity:** 0.984556902<br>**Text:** 2 . người điều_khiển , người ngồi trên xe mô_tô , xe gắn_máy ( kể_cả xe_máy_điện ) , xe_đạp máy khi tham_gia giao_thông vi_phạm_quy_định về đội mũ bảo_hiểm thì bị xử_phạt vi_phạm hành_chính theo quy_định của chính_phủ về xử_phạt vi_phạm hành_chính trong lĩnh_vực giao_thông đường_bộ .<br>**Metadata:** {'filename': 'Thông_tư_liên_tịch_06_2013_TTLT-BKHCN-BCT-BCA-BGTV.json', 'source': '06/2013/TTLT-BKHCN-BCT-BCA-BGTVT, Chương 2, Điều 10, Khoản 2', 'len_tokenizer': 52, 'parent_text': '2. Người điều khiển, người ngồi trên xe mô tô, xe gắn máy (kể cả xe máy điện), xe đạp máy khi tham gia giao thông vi phạm quy định về đội mũ bảo hiểm thì bị xử phạt vi phạm hành chính theo quy định của Chính phủ về xử phạt vi phạm hành chính trong lĩnh vực giao thông đường bộ.'}<br>

**Node ID:** 4cd7ef9f-4085-4e9c-82c8-ddc7de961050<br>**Similarity:** 0.984724814<br>**Text:** b ) phạt tiền đến 15.000.000 đồng đối_với hành_vi vi_phạm trong lĩnh_vực giao_thông đường_bộ , đường_sắt ; b ) phạt tiền đến 75.000.000 đồng đối_với hành_vi vi_phạm trong lĩnh_vực giao_thông đường_bộ , đường_sắt ; 
 o ) chở người ngồi trên xe không đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách , trừ trường_hợp chở người_bệnh đi cấp_cứu , trẻ_em dưới 06 tuổi , áp_giải người có hành_vi vi_phạm_pháp_luật . ” ; 
 c ) sửa_đổi khoản 5 điều 6 như sau : “ 5 . phạt tiền từ 1.000.000 đồng đến 2.000.000 đồng đối_với người điều_khiển xe thực_hiện một trong các hành_vi vi_phạm sau đây : c ) điều_khiển xe ô_tô tải ( kể_cả rơ_moóc và sơ_mi rơ_moóc ) có kích_thước thùng_xe không đúng với thông_số kỹ_thuật được ghi trong giấy chứng_nhận kiểm_định an_toàn kỹ_thuật và bảo_vệ môi_trường của xe ; c ) thực_hiện_hành_vi quy_định tại điểm a khoản 3 điều này bị tịch_thu giấy_phép lưu_hành đã hết giá_trị sử_dụng hoặc không do...<br>**Metadata:** {'filename': 'Nghị_định_123_2021_NĐ-CP_sửa_đổi_các_Nghị_định_quy.json', 'source': '123/2021/NĐ-CP, Điều 2', 'len_tokenizer': 16663, 'parent_text': 'đường bộ; b) Không nhường đường hoặc gây cản trở xe được quyền ưu tiên đang phát tín hiệu ưu tiên đi làm nhiệm vụ.”; b) Điều khiển xe (kể cả rơ moóc và sơ mi rơ moóc) không gắn biển số (đối với loại xe có quy định phải gắn biển số); b) Người điều khiển xe ô tô, máy kéo và các loại xe tương tự xe ô tô không có hoặc không mang theo Giấy chứng nhận bảo hiểm trách nhiệm dân sự của chủ xe cơ giới còn hiệu lực. b) Thực hiện hành vi quy định tại khoản 2, khoản 3 Điều này bị tước quyền sử dụng Giấy phép lái xe từ 02 tháng đến 04 tháng; b) Điều khiển xe có Giấy phép lưu hành còn giá trị sử dụng nhưng tổng trọng lượng (khối lượng toàn bộ) của xe hoặc tải trọng trục xe (bao gồm cả hàng hóa xếp trên xe nếu có) vượt quá quy định trong Giấy phép lưu hành; b) Lưu hành phương tiện trên lãnh thổ Việt Nam quá thời hạn quy định dưới 30 ngày. b) Phạt tiền đến 15.000.000 đồng đối với hành vi vi phạm trong lĩnh vực giao thông đường bộ, đường sắt; b) Phạt tiền đến 75.000.000 đồng đối với hành vi vi phạm trong lĩnh vực giao thông đường bộ, đường sắt;\n\to) Chở người ngồi trên xe không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách, trừ trường hợp chở người bệnh đi cấp cứu, trẻ em dưới 06 tuổi, áp giải người có hành vi vi phạm pháp luật.”;\n\tc) Sửa đổi khoản 5 Điều 6 như sau: “5. Phạt tiền từ 1.000.000 đồng đến 2.000.000 đồng đối với người điều khiển xe thực hiện một trong các hành vi vi phạm sau đây: c) Điều khiển xe ô tô tải (kể cả rơ moóc và sơ mi rơ moóc) có kích thước thùng xe không đúng với thông số kỹ thuật được ghi trong Giấy chứng nhận kiểm định an toàn kỹ thuật và bảo vệ môi trường của xe; c) Thực hiện hành vi quy định tại điểm a khoản 3 Điều này bị tịch thu Giấy phép lưu hành đã hết giá trị sử dụng hoặc không do cơ quan có thẩm quyền cấp.'}<br>

**Node ID:** 28576e60-d87f-444e-9ba9-e0229074a655<br>**Similarity:** 0.985224332<br>**Text:** 4 . phạt tiền từ 100.000 đồng đến 200.000 đồng đối_với một trong các hành_vi vi_phạm sau đây : 
 a ) điều_khiển xe lạng_lách , đánh_võng ; đuổi nhau trên đường ; 
 b ) đi xe bằng một bánh đối_với xe_đạp , xe_đạp máy ; đi xe bằng hai bánh đối_với xe xích_lô ; 
 c ) gây tai_nạn giao_thông không dừng lại , không giữ nguyên hiện_trường , bỏ trốn không đến trình_báo với cơ_quan có thẩm_quyền , không tham_gia cấp_cứu người bị nạn ; 
 d ) người điều_khiển , người ngồi trên xe_đạp máy không đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách khi tham_gia giao_thông trên đường_bộ ; 
 đ ) chở người ngồi trên xe_đạp máy không đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” hoặc đội “ mũ bảo_hiểm cho người đi mô_tô , xe_máy ” không cài quai đúng quy_cách , trừ trường_hợp chở người_bệnh đi cấp_cứu , trẻ_em dưới 06 tuổi , áp_giải người có hành_vi vi_phạm<br>**Metadata:** {'filename': 'Nghị_định_46_2016_NĐ-CP_quy_định_xử_phạt_vi_phạm_h.json', 'source': '46/2016/NĐ-CP, Chương II, Điều 8', 'len_tokenizer': 873, 'parent_text': 'đẩy làm quầy hàng lưu động trên đường, gây cản trở giao thông;\n\tđ) Không nhường đường cho xe đi trên đường ưu tiên, đường chính từ bất kỳ hướng nào tới tại nơi đường giao nhau;\n\te) Xe đạp, xe đạp máy, xe xích lô chở quá số người quy định, trừ trường hợp chở người bệnh đi cấp cứu;\n\tg) Xếp hàng hóa vượt quá giới hạn quy định, không bảo đảm an toàn, gây trở ngại giao thông, che khuất tầm nhìn của người điều khiển;\n\th) Không chấp hành hiệu lệnh của đèn tín hiệu giao thông.\n3. Phạt tiền từ 80.000 đồng đến 100.000 đồng đối với một trong các hành vi vi phạm sau đây:\n\ta) Điều khiển xe đạp, xe đạp máy buông cả hai tay; chuyển hướng đột ngột trước đầu xe cơ giới đang chạy; dùng chân điều khiển xe đạp, xe đạp máy;\n\tb) Không chấp hành hiệu lệnh, chỉ dẫn của người điều khiển giao thông hoặc người kiểm soát giao thông;\n\tc) Người điều khiển hoặc người ngồi trên xe bám, kéo, đẩy xe khác, vật khác, mang vác vật cồng kềnh; điều khiển xe kéo theo xe khác, vật khác.\n4. Phạt tiền từ 100.000 đồng đến 200.000 đồng đối với một trong các hành vi vi phạm sau đây:\n\ta) Điều khiển xe lạng lách, đánh võng; đuổi nhau trên đường;\n\tb) Đi xe bằng một bánh đối với xe đạp, xe đạp máy; đi xe bằng hai bánh đối với xe xích lô;\n\tc) Gây tai nạn giao thông không dừng lại, không giữ nguyên hiện trường, bỏ trốn không đến trình báo với cơ quan có thẩm quyền, không tham gia cấp cứu người bị nạn;\n\td) Người điều khiển, người ngồi trên xe đạp máy không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách khi tham gia giao thông trên đường bộ;\n\tđ) Chở người ngồi trên xe đạp máy không đội “mũ bảo hiểm cho người đi mô tô, xe máy” hoặc đội “mũ bảo hiểm cho người đi mô tô, xe máy” không cài quai đúng quy cách, trừ trường hợp chở người bệnh đi cấp cứu, trẻ em dưới 06 tuổi, áp giải người có hành vi vi phạm'}<br>

**Node ID:** a1229abe-ee10-4bf6-a603-98998d491858<br>**Similarity:** 0.985845295<br>**Text:** điều 3 . quy_định về mũ bảo_hiểm , kinh_doanh mũ bảo_hiểm 
 1 . mũ bảo_hiểm cho người điều_khiển , người ngồi trên xe mô_tô , xe gắn_máy ( kể_cả xe_máy_điện ) , xe_đạp máy đội khi tham_gia giao_thông ( sau đây viết tắt là mũ bảo_hiểm ) là mũ có đủ các tính_năng sau : 
 a ) có cấu_tạo đủ 3 bộ_phận : vỏ mũ , đệm hấp_thụ xung_động bên trong vỏ mũ ( đệm bảo_vệ ) và quai đeo ; có kiểu_dáng đáp_ứng yêu_cầu quy_định tại khoản 2 điều 4 của thông_tư liên_tịch này ; 
 b ) đã được chứng_nhận hợp quy , công_bố hợp quy phù_hợp quy_chuẩn kỹ_thuật quốc_gia qcvn 2 : 2008 / bkhcn ban_hành kèm theo quyết_định số 04 / 2008 / qđ - bkhcn ngày 28 tháng 4 năm 2008 của bộ_trưởng bộ khoa_học và công_nghệ ; được gắn dấu hợp quy cr , ghi nhãn hàng_hóa theo quy_định của pháp_luật . 
 2 . kinh_doanh mũ bảo_hiểm gồm các hoạt_động : bán_buôn , bán_lẻ , đại_lý mũ bảo_hiểm tại việt_nam .<br>**Metadata:** {'filename': 'Thông_tư_liên_tịch_06_2013_TTLT-BKHCN-BCT-BCA-BGTV.json', 'source': '06/2013/TTLT-BKHCN-BCT-BCA-BGTVT, Chương 1, Điều 3', 'len_tokenizer': 193, 'parent_text': 'Điều 3. Quy định về mũ bảo hiểm, kinh doanh mũ bảo hiểm\n1. Mũ bảo hiểm cho người điều khiển, người ngồi trên xe mô tô, xe gắn máy (kể cả xe máy điện), xe đạp máy đội khi tham gia giao thông (sau đây viết tắt là mũ bảo hiểm) là mũ có đủ các tính năng sau:\n\ta) Có cấu tạo đủ 3 bộ phận: vỏ mũ, đệm hấp thụ xung động bên trong vỏ mũ (đệm bảo vệ) và quai đeo; có kiểu dáng đáp ứng yêu cầu quy định tại khoản 2 Điều 4 của Thông tư liên tịch này;\n\tb) Đã được chứng nhận hợp quy, công bố hợp quy phù hợp quy chuẩn kỹ thuật quốc gia QCVN 2 : 2008/BKHCN ban hành kèm theo Quyết định số 04/2008/QĐ-BKHCN ngày 28 tháng 4 năm 2008 của Bộ trưởng Bộ Khoa học và Công nghệ; được gắn dấu hợp quy CR, ghi nhãn hàng hóa theo quy định của pháp luật.\n2. Kinh doanh mũ bảo hiểm gồm các hoạt động: bán buôn, bán lẻ, đại lý mũ bảo hiểm tại Việt Nam.'}<br>

: 